# Generate data for `test_correct_count_rate`

In [4]:
import importlib
import os
import pprint

from astropy.io import fits
import numpy as np

from jwst_magic.fsw_file_writer import buildfgssteps

In [2]:
def find_package_location(package):
    return importlib.util.find_spec(package).submodule_search_locations[0]


def find_repo_location(package):
    return os.path.abspath(os.path.join(find_package_location(package), os.pardir))

In [7]:
repo_location = find_repo_location(package='jwst_magic')
#__location__ = '/Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/'
__location__ = os.path.join(repo_location, 'jwst_magic', 'tests')
ROOT = 'test_buildfgssteps'
SEGMENT_INFILE = os.path.join(__location__, 'data', '{}_ALLpsfs.txt'.format(ROOT))
SELECTED_SEGS = os.path.join(__location__, 'data', '{}_regfile.txt'.format(ROOT))

In [8]:
with fits.open( os.path.join(__location__, 'data', 'fgs_data_2_cmimf.fits')) as hdulist:
    open_image = hdulist[1].data

In [12]:
d = {}
for guider in [1, 2]:
    d[guider] = {}
    for step in ['LOSTRK', 'CAL', 'ID', 'ACQ1', 'ACQ2', 'TRK']:
        d[guider][step] = {}
        
        bias = []
        norm = []
        strips = []
        cds = []
        product = []
        
        if step != 'LOSTRK':
            n = 10
        else:
            n = 1
            
        for i in range(n):
            # Run the code
            BFS = buildfgssteps.BuildFGSSteps(open_image, guider, 'test', step, 
                                              guiding_selections_file=SELECTED_SEGS,
                                              out_dir=__location__);#, catalog=SEGMENT_INFILE);

            # Don't know how to test bias - since it is so variable
            if step is not 'LOSTRK':
                bias.append([np.min(BFS.bias), np.max(BFS.bias)])

            norm.append([np.min(BFS.time_normed_im), np.max(BFS.time_normed_im)])
            if step is 'ID':
                strips.append([np.min(BFS.strips), np.max(BFS.strips)])
            if 'TRK' not in step:
                cds.append([np.min(BFS.cds), np.max(BFS.cds)])
            product.append([np.min(BFS.image), np.max(BFS.image)])
        
        print()
        print(guider, step)
        print()
        for l, name in zip([bias, norm , strips , cds , product, BFS.countrate], 
                              ['bias', 'norm' , 'strips' , 'cds' , step, 'countrate']):
            if (l != [] and step == 'LOSTRK') or (name in ['countrate', 'norm']):
                d[guider][step][name] = l[0]
            elif l != [] and name != 'countrate':
#                 print(name, ': [{}, {}]'.format(np.average([m[0] for m in l]), np.average([m[1] for m in l])))
#                 print(name)
#                 print(l)
#                 print()
#                 print(np.average([m[0] for m in l]))
#                 print(np.average([m[1] for m in l]))
                d[guider][step][name] = [round(np.average([m[0] for m in l]), -1), 
                                         round(np.average([m[1] for m in l]), -1)]
#             elif name == 'countrate':
#                 d[guider][step][name] = l
#         print('countrates:', BFS.countrate)
        

2021-06-02 12:04:07,689 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120407_buildfgssteps_test.log
2021-06-02 12:04:07,690 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:07,693 - INFO - FSW File Writing: Creating LOSTRK FSW files

1 LOSTRK

2021-06-02 12:04:07,731 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120407_buildfgssteps_test.log
2021-06-02 12:04:07,732 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file


/Users/kbrooks/miniconda2/envs/magic/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


2021-06-02 12:04:08,402 - INFO - FSW File Writing: Adjusting 282 pixels that are saturated in read 2.
2021-06-02 12:04:08,419 - INFO - FSW File Writing: Adjusting 162 pixels that are saturated in both reads.
2021-06-02 12:04:08,437 - INFO - FSW File Writing: Creating CAL FSW files
2021-06-02 12:04:08,472 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120408_buildfgssteps_test.log
2021-06-02 12:04:08,473 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:09,138 - INFO - FSW File Writing: Adjusting 281 pixels that are saturated in read 2.
2021-06-02 12:04:09,154 - INFO - FSW File Writing: Adjusting 162 pixels that are saturated in both reads.
2021-06-02 12:04:09,165 - INFO - FSW File Writing: Creating CAL FSW files
2021-06-02 12:04:09,210 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fo

/Users/kbrooks/miniconda2/envs/magic/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


2021-06-02 12:04:17,046 - INFO - FSW File Writing: Creating ID FSW files
2021-06-02 12:04:17,147 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120417_buildfgssteps_test.log
2021-06-02 12:04:17,149 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:18,422 - INFO - FSW File Writing: Creating ID FSW files
2021-06-02 12:04:18,560 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120418_buildfgssteps_test.log
2021-06-02 12:04:18,562 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:19,856 - INFO - FSW File Writing: Creating ID FSW files
2021-06-02 12:04:19,980 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/j

2021-06-02 12:04:31,088 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120431_buildfgssteps_test.log
2021-06-02 12:04:31,089 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:31,152 - INFO - FSW File Writing: Creating ACQ1 FSW files

1 ACQ1

2021-06-02 12:04:31,165 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120431_buildfgssteps_test.log
2021-06-02 12:04:31,166 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:31,217 - INFO - FSW File Writing: Creating ACQ2 FSW files
2021-06-02 12:04:31,226 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120431_buildfgssteps_test.log


2021-06-02 12:04:40,109 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:41,177 - INFO - FSW File Writing: Creating TRK FSW files
2021-06-02 12:04:41,233 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120441_buildfgssteps_test.log
2021-06-02 12:04:41,235 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:42,291 - INFO - FSW File Writing: Creating TRK FSW files
2021-06-02 12:04:42,340 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120442_buildfgssteps_test.log
2021-06-02 12:04:42,341 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding sele

2021-06-02 12:04:51,043 - INFO - FSW File Writing: Adjusting 164 pixels that are saturated in both reads.
2021-06-02 12:04:51,052 - INFO - FSW File Writing: Creating CAL FSW files

2 CAL

2021-06-02 12:04:51,101 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120451_buildfgssteps_test.log
2021-06-02 12:04:51,102 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:52,426 - INFO - FSW File Writing: Creating ID FSW files
2021-06-02 12:04:52,510 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120452_buildfgssteps_test.log
2021-06-02 12:04:52,511 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:04:53,791 - INFO - FSW File Writing: Creati

2021-06-02 12:05:06,344 - INFO - FSW File Writing: Creating ACQ1 FSW files
2021-06-02 12:05:06,352 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120506_buildfgssteps_test.log
2021-06-02 12:05:06,353 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:05:06,429 - INFO - FSW File Writing: Creating ACQ1 FSW files
2021-06-02 12:05:06,442 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120506_buildfgssteps_test.log
2021-06-02 12:05:06,443 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:05:06,517 - INFO - FSW File Writing: Creating ACQ1 FSW files

2 ACQ1

2021-06-02 12:05:06,525 - INFO - Started logging to file /Users/kbrooks/git_repos/jw

2021-06-02 12:05:14,462 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120514_buildfgssteps_test.log
2021-06-02 12:05:14,463 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:05:15,582 - INFO - FSW File Writing: Creating TRK FSW files
2021-06-02 12:05:15,636 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120515_buildfgssteps_test.log
2021-06-02 12:05:15,638 - INFO - FSW File Writing: Using /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2021-06-02 12:05:16,711 - INFO - FSW File Writing: Creating TRK FSW files
2021-06-02 12:05:16,756 - INFO - Started logging to file /Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic/logs/2021_06_02_Wed_120516_buildfgssteps_test.log
2021-06-02 

In [13]:
pprint.pprint(d)

{1: {'ACQ1': {'ACQ1': [7140.0, 26160.0],
              'bias': [7140.0, 26160.0],
              'cds': [-20.0, 4210.0],
              'countrate': 65596.0,
              'norm': [0.0039671236, 1977.3064]},
     'ACQ2': {'ACQ2': [9170.0, 16610.0],
              'bias': [9170.0, 16610.0],
              'cds': [-20.0, 630.0],
              'countrate': 65596.0,
              'norm': [0.02249858, 137.2947]},
     'CAL': {'CAL': [1210.0, 65540.0],
             'bias': [1210.0, 30270.0],
             'cds': [-20.0, 65000.0],
             'countrate': 65596.0,
             'norm': [0.0, 166565.97]},
     'ID': {'ID': [1210.0, 30290.0],
            'bias': [1210.0, 30290.0],
            'cds': [-20.0, 5520.0],
            'countrate': 65596.0,
            'norm': [0.0, 5283.9473],
            'strips': [1210.0, 30290.0]},
     'LOSTRK': {'LOSTRK': [6.730232e-05, 1.5393685],
                'countrate': 65596.0,
                'norm': [0.0060026245, 137.2947]},
     'TRK': {'TRK': [9170.0, 166

# Generate data for `coordinate_tranforms`

In [ ]:
import inspect
from jwst_magic.utils import coordinate_transforms

In [ ]:
for func in sorted(dir(coordinate_transforms)):
    if not func.startswith('_') and '2' in func:
        print(func)
        func_to_call = getattr(coordinate_transforms, func)
#         result = func_to_call()
        arg_names = inspect.getfullargspec(func_to_call)[0]
#         print(arg_names)
    
        if 'x_raw' in arg_names:
            x, y = (1743.3, 241.9)
            
        else:
            x, y = (-55.736935, 8.139518)
            
        if 'guider' in arg_names:
            for guider in range(2):
                result = func_to_call(x, y, guider + 1)
                print(result)
        else:
            result = func_to_call(x, y)
            print(result)

In [ ]:
method_to_call = getattr(foo, 'bar')
result = method_to_call()

In [ ]:
ANGLE_COORDS = (-55.736935, 8.139518)
coordinate_transforms.DHAS2Raw(*ANGLE_COORDS, 1)